In [ ]:
# Import necessary modules.
import time
import numpy as np
import pandas as pd
from od_cal_cls.pca_spsa import pca_spsa
import plotly.graph_objects as go

In [ ]:
# Set up plotting templete.
tempelete_01_white = dict(
    layout = go.Layout(
        # Layout properties
        title_font_size= 14,
        title_x= 0.1,
        font_size= 11,
        font_color= "#000000",
        font_family= "Times New Roman",
        margin_b = 65,
        margin_l = 60,
        margin_r = 30,
        margin_t = 50,
        plot_bgcolor= "#ffffff",
        # X axis properties
        xaxis_color= "#000000",
        xaxis_linecolor= "#000000",
        xaxis_ticks= "inside",        
        xaxis_tickfont_color= "#000000",
        xaxis_tickfont_family= "Times New Roman",
        xaxis_mirror= True,
        xaxis_showline= True,
        xaxis_showgrid= False,
        # Y axis properties
        yaxis_color= "#000000",
        yaxis_linecolor= "#000000",
        yaxis_ticks= "inside",
        yaxis_tickfont_color= "#000000",
        yaxis_tickfont_family= "Times New Roman",
        yaxis_mirror= True,
        yaxis_showline= True,
        yaxis_showgrid= False,
    )
)

In [ ]:
# Visualize & Check step value.
def eval_step(x, step_ini, param_A, param_alpha):
    step = step_ini / ((x + param_A)**param_alpha)
    return step

iter_opti = list(range(1,201))
step = list(map(lambda x: eval_step(x, step_ini= 220, param_A= 50, param_alpha= 0.602), iter_opti))

fig_step = go.Figure()

fig_step.add_trace(
    go.Scatter(
        x= iter_opti,
        y= step,
        line_color = "#000000",
    )
)

fig_step.update_layout(
    title= "Step Changes",
    xaxis_title= "Number Iteration",
    yaxis_title= "Step Value [NrVeh/hr]",
    width= 500,
    height= 350,
    template= tempelete_01_white,
)

fig_step.update_xaxes(
    range= [0, 200]
)

fig_step.show()

In [ ]:
# Visualize & Check perturbation value.
def eval_perturb(x, perturb_ini, param_gamma):
    perturb = perturb_ini / (x**param_gamma)
    return perturb


iter_opti = list(range(1,201))
perturb = list(map(lambda x: eval_perturb(x, perturb_ini= 0.5, param_gamma=0.02), iter_opti))

fig_perturb = go.Figure()

fig_perturb.add_trace(
    go.Scatter(
        x= iter_opti,
        y= perturb,
        line_color = "#000000",
    )
)

fig_perturb.update_layout(
    title= "Perturbing Changes",
    xaxis_title= "Number Iteration",
    yaxis_title= "Perturbing Value [NrVeh/hr]",
    width= 500,
    height= 350,
    template= tempelete_01_white,
)

fig_perturb.update_xaxes(
    range= [0, 200]
)

fig_perturb.show()

In [ ]:
# Create SPSA object.
spsa_sumo = pca_spsa(
    in_fl_step_ini= 4, in_fl_param_a= 50, in_fl_param_alpha= 0.602,
    in_fl_perturb_ini= 0.5, in_fl_param_gamma= 0.03, 
    in_int_iter_gradi= 1, in_int_iter_opti= 1, 
    in_lv_seg_step= False, in_lv_seg_perturb= False, in_int_seg_size= 5,
)

In [ ]:
# Get iterating numbers.
int_nr_iter_opti = spsa_sumo.int_iter_opti
int_nr_iter_gradi = spsa_sumo.int_iter_gradi

In [ ]:
# Import true od matrix.
str_path_od_true = "./data_tabular/true_od_v02.csv"
spsa_sumo.read_od_true(str_path_od_true)

In [ ]:
# Import biased od matrix.
str_path_od_base = "./data_tabular/biased_od_v02.csv"
spsa_sumo.read_od_base(str_path_od_base)

In [ ]:
# Import true flow.
str_path_flow_true = "./data_tabular/edge_flow_true_v02.csv"
spsa_sumo.read_true_flow(str_path_flow_true)

In [ ]:
# Import OD samples and make PCA object. Explaining ratio is set to 0.95.
str_dir_od_samples = "data_tabular/biased_od_samples"
spsa_sumo.read_od_samples(str_dir_od_samples)
spsa_sumo.create_od_pca(in_fl_ex_var= 0.95)

In [ ]:
# Create reducted array from base OD matrix.
spsa_sumo.pca_transform()

In [ ]:
# Measure time.
start_time = time.time()
# Boost gradient?
lv_bst_gradi = False
fac_bst_gradi = 10

# LOOP_1, Optimizing Iteration.
for iter_opti in range(1, int_nr_iter_opti + 1):
    
    # Parameters update for optimisation loop.
    # Step size (a_k), Perturbation coefficient (c_k).
    spsa_sumo.param_update(iter_opti)
    
    # Define empty list to collect gradient samples.
    lst_df_gradi_tmp1 = list()
    
    # LOOP_2, Gradient Estimattion Iteration.
    for iter_gradi in range(1, int_nr_iter_gradi + 1):
        
        # Update bernoulli random dataframe.
        spsa_sumo.bernoulli_delta(iter_opti, iter_gradi)
        
        # Get reducted arr, perturbed in plus direction.
        arr_od_reduct_plus = spsa_sumo.perturbation_plus(iter_opti, iter_gradi)
        # Convert perturbed arr into dataframe again.
        df_od_perturbed_plus = spsa_sumo.pca_inverse_transform(arr_od_reduct_plus)        
        # Run simulation with perturbed OD matrix and get flow dataframe.
        df_flow_perturbed_plus = pca_spsa.sim_sumo_get_flow(iter_opti, iter_gradi, df_od_perturbed_plus)
        # Get Normalized RMSE with true flow value.
        gof_plus = pca_spsa.n_Rmse(spsa_sumo.df_true_flow, df_flow_perturbed_plus)
        print(f"        gof_plus: {gof_plus}, Iteration_{iter_opti}_{iter_gradi}.")
        
        # Get reducted arr, perturbed in minmus direction.
        arr_od_reduct_minus = spsa_sumo.perturbation_minus(iter_opti, iter_gradi)
        # Convert perturbed arr into dataframe again.
        df_od_perturbed_minus = spsa_sumo.pca_inverse_transform(arr_od_reduct_minus)
        # Run simulation with perturbed OD matrix and get flow dataframe.
        df_flow_perturbed_minus = pca_spsa.sim_sumo_get_flow(iter_opti, iter_gradi, df_od_perturbed_minus)
        # Get Normalized RMSE with true flow value.
        gof_minus = pca_spsa.n_Rmse(spsa_sumo.df_true_flow, df_flow_perturbed_minus)
        print(f"        gof_minus: {gof_minus}, Iteration_{iter_opti}_{iter_gradi}.")
        
        # Calculate sample gradient and store it in list.
        with np.errstate(divide='ignore', invalid='ignore'):
            arr_gradi_tmp2 = (gof_plus - gof_minus) / (2*spsa_sumo.fl_perturb*spsa_sumo.arr_bernoulli)
        lst_df_gradi_tmp1.append(arr_gradi_tmp2)
    
    # Calculate mean value of smaple gradients.
    arr_gradi_mean_tmp1 = sum(lst_df_gradi_tmp1) / len(lst_df_gradi_tmp1)
    if lv_bst_gradi:
        arr_gradi_mean_tmp1 = arr_gradi_mean_tmp1 * fac_bst_gradi
    spsa_sumo.lst_gradi.append(arr_gradi_mean_tmp1)
    print(f"    Gradient estimation is ready. Iteration_{iter_opti}")
    
    # Minimizing with segmented step size value.
    # OD matrix will be adjusted in opposite direction of loss funtion gradient.
    spsa_sumo.minimization_loss(iter_opti)
    spsa_sumo.df_od = spsa_sumo.pca_inverse_transform(spsa_sumo.arr_od_reduct)    
    df_flow_min = pca_spsa.sim_sumo_get_flow(iter_opti, 999, spsa_sumo.df_od)
    nRmse_min = pca_spsa.n_Rmse(spsa_sumo.df_true_flow, df_flow_min)
    spsa_sumo.lst_nRmse.append(nRmse_min)
    
    if iter_opti == 1:
        spsa_sumo.nRmse_best = nRmse_min
        spsa_sumo.df_flow_best = df_flow_min
        spsa_sumo.df_od_best = spsa_sumo.df_od
    else:
        if nRmse_min < spsa_sumo.nRmse_best:
            spsa_sumo.nRmse_best = nRmse_min
            spsa_sumo.df_flow_best = df_flow_min
            spsa_sumo.df_od_best = spsa_sumo.df_od
    
    # Make time stamp.
    time_epoch = time.time() - start_time
    spsa_sumo.lst_time_epoch.append(int(time_epoch))
    
    print(f"    Iteration_{iter_opti}/{int_nr_iter_opti}: nRMSE {nRmse_min}")
    print(f"    Best nRMSE so far: {spsa_sumo.nRmse_best}")